In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('Healthcare_dataset.xlsx', engine='openpyxl', sheet_name='Dataset', na_values=['Uknown', 'Other/Unknown'])

In [4]:
# Counting rows with NA
na_count = data.shape[0] - data.dropna().shape[0]
percent_na = na_count / data.shape[0] * 100
print("Percent of rows with NAs = ", percent_na, "%")

Percent of rows with NAs =  4.526869158878505 %


In [5]:
#Extract rows with NA values only
nan_df = data[data.isna().any(axis=1)]
#Count frequency of persistency flag
persist_na = nan_df['Persistency_Flag'].value_counts()['Persistent']
persist_tot = data['Persistency_Flag'].value_counts()['Persistent']

print('How many persistent flags will be removed by deletion method: ', persist_na / persist_tot * 100, '%')

How many persistent flags will be removed by deletion method:  4.422032583397983 %


In [6]:
#Map values to numbers
columns_map = {}
df = data.copy()
for col in df.columns:
    if str(df[col].dtype) == 'object':
        val_map = {}
        for idx, val in enumerate(list(df[col].unique())):
            if df[col].notna().iloc[idx] == True:
                val_map[val] = idx 
            columns_map[col] = val_map
print(columns_map['Race'])

{'Caucasian': 0, 'Asian': 1, 'African American': 3}


In [8]:
#dealing with missing data
from sklearn.impute import KNNImputer

df = data.copy()
#Using Mapping to encode data
for col in df.columns:
    if str(df[col].dtype) == 'object':
        df[col] = df[col].map(columns_map[col])
   

imputer= KNNImputer(n_neighbors=5)
df_filled = df.copy()

for col in df_filled.columns:
    dummy = df_filled.copy()
    #If that column has any missing values
    if df_filled[col].isna().any() == True:
        #Get indexes of missing values
        idx_missing_before = list(df_filled[col][df_filled[col].isna()].index)
        #Drop Na from other columns to be use KNN imputer
        dummy.dropna(subset=dummy.columns.difference([col]), inplace=True)
        
        if dummy[col].isna().any() == True: 
            #Get indexes of missing values after dropping rows above
            idx_missing_after = list(dummy[col][dummy[col].isna()].index)
            #Impute missing values and subtitute them in df_filled           
            dummy = imputer.fit_transform(dummy)  
            dummy = pd.DataFrame(dummy, columns=df_filled.columns).apply(round)  #Convert array from imputer() to pandas
            df_filled[col][idx_missing_before] = dummy[col][idx_missing_after]

<ipython-input-8-a36fa6ff6cdc>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filled[col][idx_missing_before] = dummy[col][idx_missing_after].copy()
